In [1]:
import cv2
import numpy as np
import torch

In [2]:
# Show image function for later use
def showImage(img, name=None):
    if not name:
        cv2.imshow("Image display", img)
    else:
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Save image function
def saveImage(filename, img, dir):
    # Get full path
    full_path = f"{dir}/{filename}"
    cv2.imwrite(full_path, img)
    print(f"Image saved to {full_path}")

In [3]:
import os
from PIL import Image

images = []
for filename in os.listdir("./data/CATANIST dataset"):
    file_path = os.path.join("./data/CATANIST dataset", filename)
    try:
        img = Image.open(file_path)
        images.append(img)
    except IOError:
        print(f"Error opening or reading image {filename}")
        
for img in images:
    print(img)
    showImage(img)

FileNotFoundError: [Errno 2] No such file or directory: './images/CATANIST dataset'